In [1]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings
from parse_custom_report import *
from yandex_disk_func import *
from db_func import *
import re
from one_target_parse_func import *

# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ 

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [3]:
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'
conn = pyodbc.connect(conn_str_local)
cursor = conn.cursor()

In [4]:
# создаем словарь с SQL запросами, чтобы в дальнейшем использовать их для получения данных из ТВ индекс
# и очистки таблиц
db_dicts = {}
db_dicts_query = {
    'group_statistic_video_report': """select source, product, report_type, format_type, max(date) as max_db_date 
    from video_stat_table group by source, product, report_type, format_type""", 
    'group_statistic_banner_report': """select source, product, report_type, format_type, max(date) as max_db_date 
    from banner_stat_table group by source, product, report_type, format_type""", 
    'video_views_report': "select * from one_target_video_views"
}

In [5]:
# забираем из БД 3 имеющиеся таблицы. Нас интересует максимальная дата
# таким образом можно понять, какие данные являются новыми и их нужно добавить в БД 

if cursor:
    print('Connect success')

for key, new_query in db_dicts_query.items():
    query = new_query
    
    warnings.simplefilter(action='ignore', category=UserWarning)
    df = pd.read_sql(query, conn)
    db_dicts[key] = df


conn.close()

print('Загрузка завершена успешно')

Connect success
Загрузка завершена успешно


In [6]:
# отдельно по таблице с Видео 
# по каждому источнику, продукту, типу отчета(базовый/бонусный) мы получаем максимальную дату из БД
db_dicts['group_statistic_video_report']['max_db_date'] = pd.to_datetime(db_dicts['group_statistic_video_report']['max_db_date']).dt.date # приводим к дате

# отдельно по таблице с Баннерами
# по каждому источнику, продукту, типу отчета(базовый/бонусный) мы получаем максимальную дату из БД
db_dicts['group_statistic_banner_report']['max_db_date'] = pd.to_datetime(db_dicts['group_statistic_banner_report']['max_db_date']).dt.date # приводим к дате

# Отдельно по досмотрам
# забираем из БД накопленную статистику по досмотрам
# дальше по каждому источнику и продукту мы можем расчитать приррост, начиная с новых дат, которых не было до того в БД
db_dicts['video_views_report'] = db_dicts['video_views_report'].rename(columns={'min_date': 'min_db_date', 'max_date': 'max_db_date'})

In [7]:
db_dicts['group_statistic_video_report'].head(3)

,source,product,report_type,format_type,max_db_date
0,beeline,15s,base,video,2024-09-22
1,beeline,6s,base,video,2024-08-17
2,beeline,6s_v1,base,video,2024-09-22


In [8]:
db_dicts['group_statistic_banner_report'][db_dicts['group_statistic_banner_report']['source']=='gnezdo']

,source,product,report_type,format_type,max_db_date


In [9]:
db_dicts['video_views_report']

,source,product,report_type,start_period,end_period,min_db_date,max_db_date,video_views_25,video_views_50,video_views_75,video_views_100
0,one target,6s,base,2024-07-17,2024-08-11,2024-07-17,2024-07-27,440581.0,401255.0,389087.0,349872.0
1,one target,6snew,base,2024-07-17,2024-08-11,2024-08-01,2024-08-17,825822.0,787462.0,708761.0,670913.0
2,one target,15s,base,2024-07-17,2024-08-11,2024-07-17,2024-07-27,438002.0,399471.0,301982.0,289762.0
3,one target,15snew,base,2024-07-17,2024-08-11,2024-08-01,2024-08-17,785784.0,722984.0,645332.0,560999.0
4,one target,6s,bonus,2024-07-17,2024-07-31,2024-07-17,2024-07-27,45170.0,40098.0,38114.0,35726.0
5,one target,6snew,bonus,2024-07-17,2024-07-31,2024-07-28,2024-08-17,72764.0,67652.0,62970.0,58765.0
6,one target,15s,bonus,2024-07-17,2024-07-31,2024-07-17,2024-07-27,47002.0,43870.0,37665.0,30989.0
7,one target,15snew,bonus,2024-07-17,2024-07-31,2024-07-28,2024-08-17,79013.0,65764.0,60991.0,56002.0


In [10]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
report_video_dict = {}
report_banner_dict = {}
report_video_views_dict = {}

In [11]:
# вызываем функцию, которая проходит по файлам в определенной папке на Яндекс диске
# если в файл эксель, то она его парсит
# в зависимости от типа креатива (Видео, Баннер или агрегированные просмотры)
# статистика сохраняется в соответсвующий словарь
get_data_from_ya_folder(yandex_folders, main_folder, file_path, yandex_token,
                                              report_video_dict, report_banner_dict, report_video_views_dict, flag='regular')

/01_new_sources_files
/02_regular_reports
astralab_banner
avito_banner
beeline_video
gnezdo_banner
hybrid_video_banner
mediaserfer_banner
turbotarget_banner


In [12]:
# если есть статистика по Баннерам, то запускаем блок обработки таких данных
if report_banner_dict:
    # создаем общий датаФрейм со статистикой по Баннерам из всех источников
    new_statistics_banner_df = pd.concat(report_banner_dict, ignore_index=True)
    
    # добавляем к этому датаФрейму максимальную дату из нашей БД
    # таким образом мы модем понять, какие даты уже загружены в БД и нам НЕ нужно их загружать
    # т.е. сейчас нам нужно оставить только новые данные
    new_statistics_banner_df  = new_statistics_banner_df.merge(db_dicts['group_statistic_banner_report'], how='left',
                                                               left_on=['source', 'product', 'report_type', 'format_type'],
                                                               right_on=['source', 'product', 'report_type', 'format_type'])
    
    new_statistics_banner_df  = new_statistics_banner_df.fillna('')
    
    # вызываем функцию для определения новых дат и старых дат
    new_statistics_banner_df['clean_flag'] = new_statistics_banner_df.apply(get_clean_flag, axis=1)
    
    # отфильтровываем датаФрейм и оставляем только новые даты
    new_statistics_banner_df = new_statistics_banner_df[new_statistics_banner_df['clean_flag']==1]

    # и также сохраняем статистику по баннерам в датаФрейм с другим названием(как в скрипте Первая загрузка)
    # просто, чтобы наследовать нейминг
    if 'views' not in new_statistics_banner_df.columns.tolist(): # Проверяем, если этого поля нет в датаФрейме, то создаем его
        new_statistics_banner_df['views'] = 0

    if 'reach' not in new_statistics_banner_df.columns.tolist(): # Проверяем, если этого поля нет в датаФрейме, то создаем его
        new_statistics_banner_df['reach'] = 0

    if 'budget' not in new_statistics_banner_df.columns.tolist(): # Проверяем, если этого поля нет в датаФрейме, то создаем его
        new_statistics_banner_df['budget'] = 0
        
    union_banner_df = new_statistics_banner_df[['source', 'product', 'report_type', 'format_type', 'date', 
                                                'reach', 'impressions', 'clicks', 'views', 'budget']]
    # if 'views' not in union_banner_df.columns.tolist(): # Проверяем, если этого поля нет в датаФрейме, то создаем его
    #     union_banner_df['views'] = 0
    union_banner_df = union_banner_df.fillna(0)
    union_banner_df['budget'] = union_banner_df['budget'].replace('', 0)
    union_banner_df['views'] = union_banner_df['views'].apply(lambda x: 0 if x=='' else x)

    # вызываем функцию нормализации данных
    union_banner_df = normalizeDataTypes(union_banner_df, 
                                        table_type='banner') 
    
    # записываем статистику в БД
    downloadTableToDB(server_local, database_local, 'banner_stat_table', union_banner_df)


Connect success
Скрипт запущен 2024-10-08 11:46:34.989235
Данные добавлены в БД: NL_db_27082024, таблица: banner_stat_table
Скрипт отработал 2024-10-08 11:46:35.010238
Время выполнения задачи: 0:00:00.021003
Загрузка завершена. Данные успешно добавлены в БД: NL_db_27082024
##################################################



In [13]:
# если есть файлы со статистикой по Видео объявленияем, то запускаем блок обработки этой статистики
if report_video_dict:
    # создаем общий датаФрейм со статистикой по видео из всех источников
    new_statistics_video_df = pd.concat(report_video_dict, ignore_index=True)
    
    # добавляем к этому датаФрейму максимальную дату из нашей БД
    # таким образом мы модем понять, какие даты уже загружены в БД и нам НЕ нужно их загружать
    # т.е. сейчас нам нужно оставить только новые данные
    new_statistics_video_df  = new_statistics_video_df.merge(db_dicts['group_statistic_video_report'], how='left',
                                                               left_on=['source', 'product', 'report_type', 'format_type'],
                                                               right_on=['source', 'product', 'report_type', 'format_type'])
    
    new_statistics_video_df  = new_statistics_video_df.fillna('')
    
    # вызываем функцию для определения новых дат и старых дат
    new_statistics_video_df['clean_flag'] = new_statistics_video_df.apply(get_clean_flag, axis=1)
    
    # отфильтровываем датаФрейм и оставляем только новые даты
    new_statistics_video_df = new_statistics_video_df[new_statistics_video_df['clean_flag']==1]

    # т.к. в эксель файле по источнику One Target нет статистики досмотров (ее нужно считать отдельно)
    # мы выделяем из справочника распарсенных фалойв One Target (базовый отчет и бонусы) в отдельный датаФрейм
    one_target_df = new_statistics_video_df[new_statistics_video_df['source']=='one target']
    # оставляем только нужные поля для источника One Target
    # в даьнейшем мы расчитаем досмотры по дням для этого источника и добавим их к этой таблице
    one_target_df = one_target_df[['date', 'source', 'product', 'report_type', 'impressions', 'reach', 'clicks', 'format_type']]
    
    # создаем отдельный датаФрейм для статистики по всем остальным источникам
    others_video_statistics_df = new_statistics_video_df[new_statistics_video_df['source']!='one target']
    # удаляем не нужные поля из таблицы с остальными источниками
    others_video_statistics_df.drop(['max_db_date', 'clean_flag'], axis=1, inplace=True)
    
    # если есть статистика по досмотрам, то нормализуем данные и заливаем в БД
    if report_video_views_dict:
        # берем НОВУЮ накопительную статистику  и сохраняем ее в отдельный датаФрейм
        one_target_views_df = pd.concat(report_video_views_dict, ignore_index=True)
    
        # сохраняем в отдельный датаФрейм НАКОПИТЕЛЬНУЮ статистику из БД
        db_video_views_df = db_dicts['video_views_report'][['source', 'product', 'report_type', 'min_db_date', 'max_db_date', 
                                                  'video_views_25', 'video_views_50', 'video_views_75', 'video_views_100']]
        if not one_target_df.empty and report_video_views_dict:
            one_target_df, one_target_views_df = get_anoter_download_one_target_data(one_target_df, one_target_views_df, db_video_views_df)
            
            # вызываем функцию нормализации данных и прямо в нее передает функцию с объединением датаФрейма
            one_target_views_df = normalizeDataTypes(one_target_views_df, 
                                                table_type='video_views')
            
            # меняем названия полей
            one_target_views_df = one_target_views_df.rename(columns={'25': 'video_views_25', '50': 'video_views_50', 
                                                                      '75': 'video_views_75', '100': 'video_views_100'
            })
        
            # вызываем функцию для добавления в БД остальных таблиц - справочников и тд
            baseDownloadDBTable(server_local, database_local, one_target_views_df, 'one_target_video_views', flag='replace')

    union_video_df = pd.concat([others_video_statistics_df, one_target_df], ignore_index=True)
    
    if 'budget' not in union_video_df.columns.tolist(): # Проверяем, если этого поля нет в датаФрейме, то создаем его
        union_video_df['budget'] = 0    

    union_video_df['budget'] = union_video_df['budget'].replace('', 0)
    # вызываем функцию нормализации данных и прямо в нее передает функцию с объединением датаФрейма
    union_video_df = normalizeDataTypes(union_video_df, table_type='video')
    
    # приводим название полей к итоговому стандарту
    union_video_df = union_video_df.rename(columns={'25': 'video_views_25', '50': 'video_views_50',
                                                   '75': 'video_views_75', '100': 'video_views_100'})
    
    # записываем статистику в БД
    downloadTableToDB(server_local, database_local, 'video_stat_table', union_video_df)

Connect success
Скрипт запущен 2024-10-08 11:46:35.053912
Данные добавлены в БД: NL_db_27082024, таблица: video_stat_table
Скрипт отработал 2024-10-08 11:46:35.067927
Время выполнения задачи: 0:00:00.014015
Загрузка завершена. Данные успешно добавлены в БД: NL_db_27082024
##################################################



In [14]:
%reset -f